# Showcase of the new LNHR DAC II QCoDeS driver (v0.2.0)

Copyright (c) Basel Precision Instruments GmbH (2025), written for the QCoDeS driver `Baspi_Lnhrdac2.py`, v0.2.0

...........................................................................................................................................................................................................................................

This notebook shows brief examples on the changes made and on how to use the new and improved QCoDeS driver for the Basel Precision Instruments LNHRDAC II.

## 1 - Imports and setting up a Station

What's new? Generally names have been changed to previous versions of the driver to fall in line with the QCoDeS recommendations. The behaviour of the DAC upon device creation has also changed. The DAC tells the user what it does after Startup.

The main driver class is the `BaspiLnhrdac2` class. For simple use cases as applying DC-voltages, this is the only class needed. If advanced functions of the DAC such as the Arbitrary Waveform Generator (AWG) or the fast adaptive 2D scan are used, the classes `BaspiLnhrdac2SWGConfig` and `BaspiLnhrdac2Fast2dConfig` are also needed.

In [1]:
from qcodes.station import Station
from Baspi_Lnhrdac2 import BaspiLnhrdac2, BaspiLnhrdac2SWGConfig, BaspiLnhrdac2Fast2dConfig

In [2]:
station = Station()
DAC = BaspiLnhrdac2('LNHRDAC', 'TCPIP0::192.168.0.5::23::SOCKET')
station.add_component(DAC)


Connected to: Basel Precision Instruments GmbH (BASPI) LNHR DAC II (SP1060) - 24 channel version (serial:SN 10600000053, firmware:Revision 3.4.9u) in 6.50s
All channels have been turned off (1 MOhm Pull-Down to AGND) upon initialization and are pre-set to 0.0 V if turned on without setting a voltage beforehand.



'LNHRDAC'

## 2 - Using the DAC as a DC Voltage source

What's new? Everything is using easy to understand parameters now.

Using the DAC as a computer controlled DC voltage source, every channel of the DAC can be controlled through three main parameters.
- `chN.voltage`: Sets or gets the voltage currently applied to the output. Default is 0.0 V.
- `chN.high_bandwidth`: Sets or gets if the high bandwidth mode (100 kHz) is activated or not. Default is high bandwidth deactivated, meaning the channel is in low bandwidth mode (100 Hz), for best noise performance.
- `chN.enable`: Enables or disables the output. If a DAC channel is disabled, no voltage is outputted (output open, 1 MOhm to AGND).

The `chN.high_bandwidth.set()` and `chN.enable.set()` can not only be controlled with the arguments `"on"` and `"off"` but with `True` and `False` too.

Using the more advanced functions of the DAC such as the AWG, the `chN.high_bandwidth` and `chN.enable` are still used to control the bandwidth and enabling or disabling the outputs.

In [ ]:
DAC.ch14.voltage.set(5.86)
DAC.ch14.high_bandwidth.set("off")
DAC.ch14.enable.set(True)

voltage = DAC.ch14.voltage.get()
print(f"{voltage} V")

In [ ]:
DAC.all.voltage.set(-1.248)
DAC.all.enable.set(True)
voltages = DAC.all.voltage.get()
print(voltages)

## 3 - Using the DAC as an Arbitrary Waveform Generator (AWG)

What's new? Again, everything is using easy to understand parameters now. Everything shown in this chapter, could have been done with the older drivers, but it was a very tedious and much more complex process and the documentation was not easily available. With the new parameters, everything has been drastically simplified.

Broadly speaking, the process of using the AWG can be divided into to steps:
- Generating and saving a waveform to the device memory 
- Outputting it on the device

The 12 channel version of the LNHR DAC II has two AWGs, AWG A and AWG B, whereas the 24 channel version has four AWGs, AWG A, AWG B, AWG C and AWG D. All AWGs have their own set of parameters.

There are two ways on how a waveform can be generated and saved to device memory.

### 3.1 - Creating a waveform using the integrated Standard Waveform Generator (SWG)

The integrated Standard Waveform Generator (SWG) instrument module aids with the generation of simple signals, such as:
- sine and cosine
- triangualar and sawtooth signals
- rectangular and pulse/ PWM signals
- fixed and random white noise

To configure the SWG, an object of class `BaspiLnhrdac2SWGConfig` must be passed to the `swg.configuration`parameter. Once parametrized, the waveform can be saved to the device memory using the `swg.apply` method.

In [ ]:
config = BaspiLnhrdac2SWGConfig(
    shape = "sawtooth",
    frequency = 80.0,
    amplitude = 1.03,
    offset = 0.0,
    phase = 0.0
)

DAC.swg.configuration.set(config)
DAC.swg.apply("A")

### 3.2 - Creating a waveform using a set of custom data

For more complex waveforms, a fully custom waveform can be directly written to device memory, using the `awgX.waveform` parameter. Before the waveform is set, the `awgX.length` parameter has to be updated. Additionally there is the option to set the `awgX.sampling_rate` parameter. It is important to note, that the AWG A and AWG B share the same sampling rate, as do AWG C and AWG D. Therefore, by changing the `awga.sampling_rate` the `awgb.sampling_rate` is changed too.

Writing a waveform directly to the device memory comes with the advantage of full customizability, however, especially relevant for larger waveforms (more points), this method is slower than using the SWG.

In [ ]:
from numpy import empty, sin
from random import random

# creating a noisy rectangular signal
waveform = empty(100)
for i in range(0, len(waveform)):
    sign = 1 if bool(round(random())) else -1
    amplitude = 8 if sin(i/10) >= 0 else -8
    waveform[i] = round(amplitude + ((random()) * sign), 6)

In [ ]:
DAC.awgb.length.set(len(waveform))
DAC.awgb.sampling_rate.set(0.1)
DAC.awgb.waveform.set(waveform)

### 3.3 - Output the saved waveform

Once the waveform is saved to the device memory, the AWG can be set up to output the waveform.

If an external trigger is desireable, the `awgX.trigger` parameter provides the following possibilities:
- `"disable"`: no external trigger, AWG started by software
- `"start only"`: rising edge on the AWGs trigger input starts the AWG
- `"start stop"`: rising edge on the AWGs trigger input starts the AWG, falling edge stops it, indefinetly repeatable
- `"single step"`: on each rising edge on the AWGs trigger input, the AWG outputs the next stored value from memory, this results in the irrelevance of the `awgX.sampling_rate` parameter

It should also be noted, that the `chN.enable` and `chN.high_bandwidth` parameters still control the channels.

In [ ]:
DAC.ch3.enable.set("on")
DAC.ch3.high_bandwidth.set("on")

DAC.awga.channel.set(3)
DAC.awga.cycles.set(615)
DAC.awga.trigger.set("disable")
DAC.awga.enable.set("on")


DAC.ch8.enable.set("on")
DAC.ch8.high_bandwidth.set("on")

DAC.awgb.channel.set(8)
DAC.awgb.cycles.set(615)
DAC.awgb.trigger.set("disable")
DAC.awgb.enable.set("on")

### 3.4 - Read and plot the saved waveform

Once saved to device memory, the user can read the waveforms through the `awgX.waveform` parameter. This is a parameter with setpoints, the setpoints are stored inside the parameter `awgX.time_axis`. Therefore the user automatically gains access to the time intervals of the waveform, which allows for an easy way to plot it.

In [ ]:
awga = DAC.awga.waveform.get()
awga_setpoints = DAC.awga.time_axis.get()

print(f"Values stored in AWG A device memory ({DAC.awga.length.get()} datapoints, in V):")
print(f"{awga}\n")

print(f"Time intervals for AWG A ({DAC.awga.length.get()} datapoints, in s):")
print(f"{awga_setpoints}\n")

In [ ]:
from qcodes.dataset import Measurement, plot_dataset

# set measurement contex
measurement = Measurement()
measurement.register_parameter(DAC.awga.time_axis)
measurement.register_parameter(DAC.awga.waveform, setpoints = (DAC.awga.time_axis,))

# get data from device and save as plottable data
with measurement.run() as datasaver:
    datasaver.add_result((DAC.awga.time_axis, DAC.awga.time_axis.get()), (DAC.awga.waveform, DAC.awga.waveform.get()))
    dataset = datasaver.dataset

plot_dataset(dataset)

## 4 - Using the DAC for 2D-scans

What's new? Similar to the AWG and SWG submodules, this driver introduces a fast adaptive 2D scan submodule with easy to understand parameters.



### 4.1 - Using the standard QCoDeS do2d

In [ ]:
from qcodes.instrument import Instrument, Parameter
from qcodes.utils.dataset.doNd import do2d
import random

# random measurement parameter mimicking a real measurement device
class RandomNumberParameter(Parameter):
    def get_raw(self):
        return random.random()

class RandomNumberInstrument(Instrument):
    def __init__(self, name):
        super().__init__(name)
        self.add_parameter('measurement', parameter_class=RandomNumberParameter, unit='V')

    def get_idn(self):
        return {"vendor": "BasPI", "model": str(self.__class__), "serial": "NA", "firmware": "NA"}

# instantiate the dummy measurement instrument
dummy_instrument = RandomNumberInstrument('random_measurement_instrument')

# select specific channels 
V1 = DAC.ch1.voltage # voltage output channel 1
V2 = DAC.ch2.voltage  # voltage output channel 2

# perform a 2D measurement using the standard QCodDes do2d function
result, _, _ = do2d(
    V1, -1, 1, 20, 0.01,
    V2, -1, 1, 20, 0.01,
    dummy_instrument.measurement,
    do_plot=True,
    show_progress=True
)


### 4.2 - Using the LNHR DAC II specific fast adaptive 2D scan 

In [ ]:
config = BaspiLnhrdac2Fast2dConfig(
        x_channel = 3,
        x_start_voltage = 0.0,
        x_stop_voltage = 0.5,
        x_steps = 10,
        y_channel = 7,
        y_start_voltage = 0.0,
        y_stop_voltage = 0.5,
        y_steps = 5,
        acquisition_delay = 0.13,
        adaptive_shift = 0.0
)

DAC.fast2d.configuration.set(config)

Starting to configure fast adaptive 2D scan. AWG A will be repurposed. AWG A and AWG B connot be used while the 2D scan is running.
Fast adaptive 2D scan sucessfully configured. Ready to start.


In [ ]:
from time import sleep

DAC.ch3.high_bandwidth.set(True)
DAC.ch7.high_bandwidth.set(True)
DAC.ch22.high_bandwidth.set(True)
DAC.ch3.enable.set(True)
DAC.ch7.enable.set(True)
DAC.ch22.enable.set(True)
DAC.fast2d.trigger_channel.set(13)
DAC.fast2d.trigger.set("point out")

# repeat 2D scans
while True:
    DAC.ch3.voltage.set(config.x_start_voltage)
    DAC.ch7.voltage.set(config.y_start_voltage)

    sleep(1)
    DAC.fast2d.enable.set("on")
    sleep(3)

In [ ]:
x_axis = DAC.fast2d.x_axis.get()
y_axis = DAC.fast2d.y_axis.get()

print(f"Y-axis values are:")
print(y_axis)
print(f"X-axis values are:")
print(x_axis)

In [ ]:
DAC.fast2d.enable.set("off")

In [ ]:
DAC.fast2d.trigger_channel.set(13)
DAC.fast2d.trigger.set("disable")